In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter1d

In [ ]:
############################################## Only need to update this cell!! ##############################################
# files Path
spike_times_file = "/Users/alexandracheng/Desktop/Harris_Lab/Collab/Wu_Sleep/Wu_Sleep_Data/spike_times.npy"
spike_clusters_file = "/Users/alexandracheng/Desktop/Harris_Lab/Collab/Wu_Sleep/Wu_Sleep_Data/spike_clusters.npy"

# parameters
bin_size = 1500
sample_rate = 30000
sigma = 6

In [ ]:
# function for calculating firing rate of each neuron(clsuters)

def firing_rate(spike_times_file, spike_clusters_file, bin_size, cluster):
    spike_times = np.load(spike_times_file)
    spike_clusters = np.load(spike_clusters_file)

    if cluster == "all":
        clust_ind = np.unique(spike_clusters)
    else:
        clust_ind = cluster
    t_end = np.max(spike_times)
    t_bins = np.arange(0,t_end,bin_size)
    
    fr = np.zeros((len(clust_ind), len(t_bins)-1))
    for i in range(len(clust_ind)):
        spikes_t_ind = spike_times[np.where(spike_clusters == clust_ind[i])[0]]
        spikes_count, edges = np.histogram(spikes_t_ind, t_bins)

        fr[i,:] = spikes_count

    return fr, t_bins[:-1]

In [ ]:
# function for smoothing firing rate via gaussian filter

def firing_rate_smooth(fr, sigma):
    fr_smooth = np.zeros(fr.shape)
    for i in range(len(fr_smooth)):
        fr_smooth[i,:] = gaussian_filter1d(fr[i,:], sigma)

    return fr_smooth

In [ ]:
fr, t_bins = firing_rate(spike_times_file, spike_clusters_file, bin_size, "all")
fr_smooth = firing_rate_smooth(fr, sigma)